In [2]:
import h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from __future__ import print_function
from matplotlib import pyplot as plt

In [ ]:
Fonction qui permet d"extraire les signaux events ou noise en trois dimention ex:(1000, 3, 12000) ainsi que le trace_name correspondant

In [3]:
def load_dataset(filename):
    with h5py.File(filename, 'r') as f:
        if 'data' in f:
            data = f['data']
            if isinstance(data, h5py.Dataset):
                print("Type du dataset 'data':", type(data))
                print("Forme du dataset 'data':", data.shape)
                data_matrix = data[:]  
                print("Matrice de données:", data_matrix)
                return data_matrix
            elif isinstance(data, h5py.Group):
                data_list = []
                trace_names = []
                for key in data.keys():
                    dataset = data[key]
                    if isinstance(dataset, h5py.Dataset):
                        # Vérifier la forme des données extraites
                        if dataset.ndim == 2:
                            
                            #print(dataset.shape[0])
                            data_array = np.zeros((1, dataset.shape[0], dataset.shape[1]), dtype=np.float32)  # Créer une matrice vide de trois dimensions
                            for i in range(2):  # Boucle à travers les canaux
                                data_array[:,i,:] = dataset[i,:]  # Récupérer les valeurs des trois canaux existants
                            data_list.append(data_array)
                            #trace_names.append(key)
                            # Concaténer les noms de clés avec un tableau de zéros
                            trace_names = np.concatenate([np.array([key] ).reshape(-1, 1) for key in data.keys()])
                            # Remodeler en un vecteur colonne
                            trace_names=trace_names.reshape(-1, 1)                          
                            #trace_names['trace_names']=trace_names
                        
                        else:
                            print(f"Les données pour la clé '{key}' ne sont pas sous forme de tableau 2D.")
                if data_list:
                    stacked_data_matrix = np.stack(data_list, axis=0)
                    #Redimensionnement des données
                    stacked_data_matrix = np.squeeze(stacked_data_matrix, axis=1)

                    # Vérification du nouveau shape
                    #print(data_resized.shape) 
                    return stacked_data_matrix, trace_names

# Utilisation de la fonction, pour charger et convertir le dataset à partir du fichier HDF5
filename1 = 'Instance_noise_1k.hdf5'
noise_matrix, trace_names_noise = load_dataset(filename1)

filename2 = 'Instance_events_counts_10k.hdf5'
events_matrix, trace_names_events = load_dataset(filename2)

#print (events_matrix.shape)

In [ ]:
# Création d'une colonne 'source_type' avec des valeurs constantes de 0
source_type_n = np.zeros((len(trace_names_noise), 1), dtype=int)

# Création d'une colonne 'source_type' avec des valeurs constantes de 1
source_type_e = np.ones((len(trace_names_events), 1), dtype=int)

# Concaténation de la colonne 'source_type' avec trace_names_noise
trace_names_n = np.concatenate((trace_names_noise, source_type_n), axis=1)
trace_names_e = np.concatenate((trace_names_events, source_type_e), axis=1)

# Vérification de la forme de la matrice résultante
print(trace_names_e)

In [ ]:
Préparation des données dentrainement, validation et de test
Concatenation de données noise et events pour chaque partie.
la meme chose est faite pour les trace_names qui vont avec les données.

In [ ]:
# Préparation des données d'entrainement, validation et de test

from torch.utils.data import TensorDataset
import torch
from torch.utils.data import DataLoader

def prepa_donnees (events_matrix, trace_names_e, noise_matrix,trace_names_n, split_size= split_size)
    # Diviser les données de bruit en ensemble d'entraînement et ensemble de test
    df_noise_train1, df_noise_test = train_test_split(noise_matrix, test_size=split_size, random_state=42)
    df_noise_train, df_noise_validation = train_test_split(df_noise_train1, test_size=split_size, random_state=42)

    trace_noise1, trace_noise_test =train_test_split(trace_names_n, test_size=split_size, random_state=42)
    trace_noise_train, trace_noise_val = train_test_split(trace_noise1, test_size=split_size, random_state=42)

    trace_events1, trace_events_test =train_test_split(trace_names_e, test_size=split_size, random_state=42)
    trace_events_train, trace_events_val = train_test_split(trace_events1, test_size=split_size, random_state=42)


    df_train1, df_test = train_test_split(events_matrix, test_size=split_size, random_state=42)
    df_train, df_validation = train_test_split(df_train1, test_size=split_size, random_state=42)


    # Concaténer df_noise_train avec df_train
    df_train_concat = np.concatenate([df_train, df_noise_train], axis=0)
    trace_train = np.concatenate([trace_events_train, trace_noise_train], axis=0)
    trace_train = trace_train[:,1].astype(int)

    # Concaténer df_noise_test avec df_test
    df_test_concat = np.concatenate([df_test, df_noise_test], axis=0)
    trace_test= np.concatenate([trace_events_test, trace_noise_test], axis=0)
    # Convertir les étiquettes en entiers
    trace_test = trace_test[:,1].astype(int)

    #print (df_test_concat)
    # Concaténer df_noise_validation avec df_validation
    df_val_concat = np.concatenate([df_validation, df_noise_validation], axis=0)
    trace_val= np.concatenate([trace_events_val, trace_noise_val], axis=0)
    trace_val = trace_val[:,1].astype(int)
    
    return df_train_concat, df_val_concat, df_test_concat, trace_train, trace_test, trace_val

df_train_concat, df_val_concat, df_test_concat, trace_train, trace_test, trace_val= prepa_donnees (events_matrix, trace_names_e, noise_matrix,trace_names_n,split_size= 0.25)


In [ ]:
def normalisation(df_train_concat, df_val_concat, df_test_concat)

    # Normaliser toute la donnée
    mean_t = np.mean(df_train_concat)
    std_t = np.std(df_train_concat)
    norm_data_t = (df_train_concat - mean_t) / std_t

    mean_v = np.mean(df_val_concat)
    std_v = np.std(df_val_concat)
    norm_data_v = (df_val_concat - mean_v) / std_v

    mean_ts = np.mean(df_test_concat)
    std_ts = np.std(df_test_concat)
    norm_data_ts = (df_test_concat - mean_ts) / std_ts
    
    return norm_data_t, norm_data_v, norm_data_ts

norm_data_t, norm_data_v, norm_data_ts= normalisation(df_train_concat, df_val_concat, df_test_concat)


